# Project Report
The project is open-ended with the exact goal to be decided amongst your group but the theme is around simulations and data relating to the coronavirus epidemic. You should make a program or several programs that can either run simulations or produce plots based on real data.

Along with your code there should be a group Project Report explaining your code and what it does and including some figures that are output from your code or perhaps links to videos or any other output.

The report should not be long. The purpose of the report is to succinctly explain any models you are using or data that you have obtained and to show the resulting plots that you have made. This project is about code (rather than modelling, data analysis, or epidemiology) so the report should be about showcasing some outputs of your code and explaining how your code can be used to produce those as well as other outputs. A minimum of explanation is needed for those outputs to be understandable but you do not need to have a "background" section or a "literature review" or anything like that - It would be reasonable to include links to any references though e.g. if your models are inspired by something else.

# Data
Carl writes where he got the data from

# RoadMap 1
We're using https://en.wikipedia.org/wiki/Erd%C5%91s%E2%80%93R%C3%A9nyi_model as the base model we're running our simulations on.
On NDlib, there are 12 Epidemic models:
In NDlib are implemented the following 6 Epidemic models:
    * SI
    * SIS
    * SIR
    * SEIR
    * SEIS
    * SWIR
And another 6 less biological models:
    * Threshold
    * Generalised Threshold
    * Kertesz Threshold
    * Profile
    * Profile-Threshold
    * Independent Cascades

The library also includes Opinion Dynamics - to help the study of how opinions form in a population taking into account various social theories. This is outside the scope of the project.

Consider setting
1. Node attributes - e.g threshold to be infected. Not applicable for all models (SEIR has none)
2. Edge attributes - also not applicable
3. Possibly doing a non-random infection seeding?

# Roadmap 1
We quickly realised that it was not feasible to create a graph with the actual values of population. Even 100,000 (1/5th of Bristol's population causes the program to freeze)

https://www.dam.brown.edu/drp/talks/JosephSheely.pdf gives us parameters to model COVID
For reproducing the kinetics of real data we made the following assumptions:
In the absence of containing strategies, the infection is characterized by a high infection rate (we take α=0.9) and a low removal rate (γ=0.025 or 0.05). (https://www.nature.com/articles/s41598-021-03721-x#Sec2)

Latent period set to 3 =https://www.pnas.org/doi/10.1073/pnas.2008087117
(TODO: CHECK!) removal rate 0.1 so that in 10 days eveyrone is removed

TODO: Figure out what different model sizes change - so far it doesn't change anything if the parameters don't change lmfaoo

In [3]:
# Measuring execution time for various graph sizes
import datetime
import networkx as nx

timings = {}
for i in (10**i for i in range(1,4)):
    start_time = datetime.datetime.now()
    nx.erdos_renyi_graph(i, 0.1)
    timings[i] = datetime.datetime.now() - start_time

for i in timings:
    print(i)
print(timings)

TypeError: cannot unpack non-iterable int object